In [4]:
from openai import OpenAI

In [5]:
client = OpenAI()

In [13]:
batch_file = client.files.create(
  file=open("../../eval_data/batch_tasks_question_generation.jsonl", "rb"),
  purpose="batch"
)

In [14]:
print(batch_file)

FileObject(id='file-8fvkqba6RDdJqKCsHY6y4x', bytes=92968399, created_at=1749949111, filename='batch_tasks_question_generation.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)


In [15]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [27]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_684e1af3725081908183e2775538c1d0', completion_window='24h', created_at=1749949171, endpoint='/v1/chat/completions', input_file_id='file-8fvkqba6RDdJqKCsHY6y4x', object='batch', status='cancelling', cancelled_at=None, cancelling_at=1749949280, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1750035571, failed_at=None, finalizing_at=None, in_progress_at=1749949175, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=108, total=115))
